In [ ]:
#default_exp model

# Base Model
> This class contains the base which is used to train data upon.

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export
from torchlife.models.km import KaplanMeier
from torchlife.models.ph import PieceWiseHazard
from torchlife.models.cox import ProportionalHazard

from torchlife.data import create_db, create_test_dl

from torchlife.losses import *

from fastai.basics import Learner

In [ ]:
# export
__text2model__ = {
    'km': KaplanMeier,
    'ph': PieceWiseHazard,
    'cox': ProportionalHazard
}

__text2loss__ = {
    'km': hazard_loss,
    'ph': hazard_loss,
    'cox': hazard_loss
}

In [ ]:
# export
class Model:
    def __init__(self, model:str, model_args:dict=None, breakpoints:list=None, 
                 bs:int=128, epochs:int=20, lr:float=1, beta:float=0):
        self.model = __text2model__[model](**model_args)
        self.loss = __text2loss__[model]
        self.breakpoints = breakpoints
        self.bs, self.epochs, self.lr, self.beta = bs, epochs, lr, beta
        
    def lr_find(df):
        db = create_db(df, self.breakpoints)
        learner = Learner(db, model, loss_func=hazard_loss)
        learner.lr_find(wd=self.beta)
        learner.recorder.plot()
        
    def fit(df):
        if hasattr(self.model, 'fit'):
            self.model.fit(df)
        else:
            db = create_db(df, self.breakpoints)
            learner = Learner(db, model, loss_func=hazard_loss)
            learner.fit(self.epochs, lr=self.lr, wd=self.beta)
            
    def predict(df):
        test_dl = create_test_dl(df)
        preds = []
        for x in test_dl:
            preds.append(self.model(x))
        return preds
        
    def plot_survival():
        self.model.plot_survival_function()

In [ ]:
# hide
from nbdev.export import *
notebook2script("model.ipynb")

Converted model.ipynb.
